In [1]:
import pandas as pd
import sql_functions as sql_int
import Levenshtein

## Get Names from Tournament


take the playerlist and reduce duplikates in it

In [2]:
df_player_import = pd.read_csv('/program/tft_overview/tft_Tables/data_sheets/name_list_GSC2.csv',
                         dtype={'liquipedia_name' : str,'coutry_flag' : str} )
df_player_import.rename(columns={'liquipedia_name' : 'import_name'},inplace=True)
df_player_import.drop_duplicates(inplace=True)

Same liquipedia name but different country correct the entrys

In [3]:
df_player_import[df_player_import.duplicated(keep = False, subset=['import_name']) == True].sort_values(by='import_name')

,import_name


#### delete double entrys

In [7]:
df_player_import.drop([148],inplace=True)


## Compare Names with *alternative_names*

In [4]:
schema = 'tft_tables'
sql_query = f'select * from {schema}.alternative_names;'
df_alternative_names = pd.DataFrame(sql_int.get_data_mysql(sql_query),
                           columns=['player_name','liquipedia_name'])

replace the imported names with liquipedia_names from the alternative list.   
If there was someone with his alternative name in, they have now liquipedia_name

In [7]:
df_merge_import_alternative = pd.merge(df_player_import, df_alternative_names, how="left", left_on='import_name',right_on='player_name')

df_merge_import_alternative.loc[df_merge_import_alternative['player_name'].notna(), 'import_name'] = df_merge_import_alternative['liquipedia_name']
df_merge_import_alternative.drop(columns=['player_name','liquipedia_name'],inplace=True)
df_merge_import_alternative

,import_name
0,5454
1,valentin3411
2,Memo
3,Kingdanzz
4,Aexoq
...,...
123,Epik
124,keenko
125,Littletoof
126,DeliryummEUW


2nd check cause 2nd merge

In [8]:
df_merge_import_alternative[df_merge_import_alternative.duplicated(keep= False, subset=['import_name']) == True].sort_values(by='import_name')

,import_name


## Search Names in *players*

In [9]:
sql_query = f'select liquipedia_name from {schema}.players;'
df_liquipedia_names = pd.DataFrame(sql_int.get_data_mysql(sql_query),
                                columns=['liquipedia_name'])


In [10]:
df_merge_import_alternatives_liquipedia =pd.merge(
                                            df_merge_import_alternative,df_liquipedia_names, how='left',
                                            left_on= 'import_name', right_on='liquipedia_name')

df_new_players are all players we put in over our names_list, which arn't in *players* 

In [20]:
df_new_players = df_merge_import_alternatives_liquipedia[df_merge_import_alternatives_liquipedia['liquipedia_name'] \
                                                         .isna()].drop(columns=['liquipedia_name'])

## Push new Names to *alternative_names*

In [12]:
list_new_players = df_new_players['import_name'].to_list()
list_liquipedia_name = df_liquipedia_names['liquipedia_name'].to_list()

 Check similarity 

In [13]:
similar_names = []
for i in list_new_players :
    for j in list_liquipedia_name:
        if i.lower() == j.lower():
            if i == j:
                pass
            else:
                similar_names.append([j,i])
        elif  Levenshtein.jaro(i,j) > 0.75:
            similar_names.append([j,i])
df_similar_names = pd.DataFrame(similar_names,columns=['liquipedia_name','import_name'])

#### Controll
***This list must be controlled before pushed!***

In [21]:
df_similar_names

,liquipedia_name,import_name


Do some magic here

In [15]:
df_similar_names.drop([0,1],inplace=True)
df_similar_names

,liquipedia_name,import_name


#### Data push
**Hier ist der Push in die Database, be sure**

The datafram has to be established

In [23]:
df_similar_names_push = []

In [24]:
df_similar_names_push.rename(columns={'import_name' : 'player_name'},inplace=True)
#sql_int.push_to_database(df_similar_names,'alternative_names',sql_int.get_engine_alchemy(),'tft_tables')


The alternative_names table was imported successfully.


## Push new names in *players*

In [23]:
df_new_players = pd.merge(df_new_players,df_similar_names, how='left',
                              left_on= 'import_name', right_on='import_name')  ## on player name cause all with same liquipedia_name are already out

df_new_players = df_new_players[df_new_players['liquipedia_name'].isna()]

df_new_players.drop(columns=['liquipedia_name'],inplace=True)

In [25]:
new_players_push = df_new_players.rename(columns={'import_name' : 'liquipedia_name','country_flag' : 'country_name'})
#sql_int.push_to_database(new_players_push,'players',sql_int.get_engine_alchemy(),'tft_tables')

The players table was imported successfully.


In [ ]:
df #updating data in dataframe wäre der dataframe mit den liquid_namen mit flagge. (die vorher keine gehabt haben)

s_update = "" #String of updations

# Loop through the data frame

for i in range(len(df)):
    s_update += "update players set country_name = '%s' where liquipedia_name = '%s';" %(df[country_name][i], df[liquipedia_name][i])